## Вопросы

### 1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec? В чем принципиальные отличия item-item рекомендаций от ALS?


Для расчетной части, item_item и user_user - матрицы, следовательно обрабатываются будут одними и теми же методами. Различаться будут собственно данными (характеристики это пользователей, или товаров). Item-item будут хорошо применимы для новых пользователей, но плохо для новых товаров. User-user наоборот хорошо для новых товаров, но плохо для новых пользователей.

ALS компактнее, т.к. большие разряженные матрицы разбиваются на две более плотных с меньшими рангами.

### 2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 

* Сезонность (Купальники зимой не так актуальны).
* Количество купленных схожих товаров.
* Количество дизлайков на товарах этой категории (если это какой-то рекомендательный сервис, например Кинопоиск или Яндекс Музыка).

### 3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)

* Сложность вычислений
* Минимизация квадрата потерь не всегда оптимальна.
* Трафик


## Практическая часть


In [1]:
!pip install implicit

In [3]:
import implicit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from implicit.evaluation import precision_at_k

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

In [5]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list) 
    
    return precision

In [12]:
DATA_PATH = '../data'

In [18]:
data = pd.read_csv(os.path.join(DATA_PATH, 'transaction_data.csv'))

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [20]:
result = pd.read_csv(os.path.join(DATA_PATH, 'predictions_basic.csv'))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,[ 829563 830156 832990 840361 856942 ...,"[826860, 9420257, 1780146, 109081, 82699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[854852, 867547, 879528, 883218, 933835]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1370778, 1000050, 1277455, 852112, 829303]","[6534178, 6533889, 1029743, 6534166, 1082185]","[847573, 862139, 985699, 999971, 1030182]"


In [21]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [22]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [23]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [25]:
csr_mat = csr_matrix(user_item_matrix).T.tocsr()

In [26]:
bm25_weight_matrix = bm25_weight(user_item_matrix.T).T  
tfidf_matrix = tfidf_weight(user_item_matrix.T).T 

In [27]:
#  CosineSimilarity
mu = user_item_matrix.mean().mean() # находим среднюю оценку всех товаров
ru_overline = user_item_matrix.mean(axis=1)# находим среднюю оценку данного пользователя
cosine_matrix = (user_item_matrix.T - ru_overline.values - mu).T

In [28]:
max_tfidf = tfidf_matrix.astype(float).max() 

cos_tfidf_matrix= (tfidf_matrix.astype(float) / max_tfidf) + np.array(cosine_matrix) 

In [33]:
matrixes = {
  'user_item':user_item_matrix, 
  'bm25':bm25_weight_matrix, 
  'tfidf':tfidf_matrix, 
  'cosine':cosine_matrix, 
  'cos&tfidf':cos_tfidf_matrix
}
regularizations = [ 0.01, 0.1, 1, 10, 100, 1000]
n_factors = [10, 15, 20, 50, 100]

In [32]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [34]:
for matrix_name in matrixes:
    for regularization in regularizations:
        for n_factor in n_factors:
          model = AlternatingLeastSquares(factors=n_factor, 
                                regularization=regularization,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

          model.fit(csr_matrix(matrixs[matrix_name]).T.tocsr(),  # На вход item-user matrix
                    show_progress=True)
          
          result[f'{matrix_name} {n_factor} factor {regularization} reg'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        

100%|██████████| 15/15 [01:22<00:00,  5.49s/it, loss=0.871]


In [35]:
report = pd.DataFrame(
    [
        (model_, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5)) for model_ in list(result)[2:]
    ], 
    columns=['model name', 'precision@k']
)

In [36]:
report.T

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
model name,user_item 10 factor 0.01 reg,user_item 15 factor 0.01 reg,user_item 20 factor 0.01 reg,user_item 50 factor 0.01 reg,user_item 100 factor 0.01 reg,user_item 10 factor 0.1 reg,user_item 15 factor 0.1 reg,user_item 20 factor 0.1 reg,user_item 50 factor 0.1 reg,user_item 100 factor 0.1 reg,...,cos&tfidf 10 factor 100 reg,cos&tfidf 15 factor 100 reg,cos&tfidf 20 factor 100 reg,cos&tfidf 50 factor 100 reg,cos&tfidf 100 factor 100 reg,cos&tfidf 10 factor 1000 reg,cos&tfidf 15 factor 1000 reg,cos&tfidf 20 factor 1000 reg,cos&tfidf 50 factor 1000 reg,cos&tfidf 100 factor 1000 reg
precision@k,0.18764,0.18383,0.18172,0.17921,0.17278,0.18714,0.18915,0.18152,0.18041,0.16605,...,0.21226,0.21226,0.21316,0.21246,0.21286,0.20944,0.20944,0.20944,0.20944,0.20944


In [37]:
report.groupby('precision@k').get_group(report['precision@k'].max())

,model name,precision@k
54,bm25 100 factor 100 reg,0.25314


# Выводы

1) ALS работает лучше, чем user-user, item-item
2) BM100 is the best при регуляризации 100!